# Zarrify DART Reanalysis History Files

In [1]:
import numpy as np
import xarray as xr
import intake
import ast
import pandas as pd
from helpers import update_metadata

import dask
import dask.distributed
from dask.distributed import Client
from ncar_jobqueue import NCARCluster
from distributed.utils import format_bytes

import fsspec
from pathlib import Path
import shutil 
import os
from functools import reduce
from operator import mul

### Configuration Parameters

In [12]:
catalog_path = './dart-zarr-input.json'

output_folder = '/glade/scratch/bonnland/DART/ds345.0/zarr-publish'

#output_variables = ['T', 'PS', 'Q', 'US', 'VS', 'CLDICE', 'CLDLIQ']

#output_variables = ['TSA', 'ER', 'EFLX_LH_TOT', 'HR']
output_variables = ['ER', 'EFLX_LH_TOT', 'HR', 'TSA']

# Number of elements per chunk in the target stores.
# A negative value means "don't chunk this dimension".
#target_chunks = {'pftlat': 32, 
#                 'pftlon': 32, 
#                 'time': 30, 
#                 'member_id': 10}

target_chunks = {'lat': 32, 
                 'lon': 32, 
                 'time': 800, 
                 'member_id': 40}


## Run These Cells for Dask CASPER

In [3]:
# Processes is processes PER CORE.
# This one works fine.
#cluster = NCARCluster(cores=15, processes=1, memory='100GB', project='STDD0003')
# This one also works, but occasionally hangs neacr the end.
#cluster = NCARCluster(cores=10, processes=1, memory='50GB', project='STDD0003')

# For Casper
num_cores = 2 #1
num_jobs = 50 #90 #4
walltime = "1:00:00" #"1:00:00"
memory = '50GB' #'50GB'

cluster = NCARCluster(cores=num_cores, processes=1, memory=memory, project='STDD0003', walltime=walltime)
cluster.scale(jobs=num_jobs)

client = Client(cluster)
cluster

## Run These Cells for Dask CHEYENNE

In [3]:
# For Cheyenne

# These are "per node", and then .scale() selects the number of nodes.
#walltime = "1:00:00"
#walltime = "00:30:00"
walltime = "00:45:00"

#  This results in about 20% maximum memory usage.
#cluster = NCARCluster(cores=1, processes=1, memory='109GB', walltime=walltime)
#num_nodes = 16


# Run 16 workers on 4 nodes, giving each worker around 25GB RAM.  
#cluster = NCARCluster(cores=4, processes=4, memory='109GB', walltime=walltime)

# # Run 4 workers on each node, giving each worker around 25GB RAM.  
# cluster = NCARCluster(cores=16, processes=4, memory='109GB', walltime=walltime)
# num_nodes = 2

# Run <= 4 workers on each node to avoid crashes.
cluster = NCARCluster(cores=10, processes=4, memory='109GB', walltime=walltime)
num_nodes = 8

cluster.scale(jobs=num_nodes)

from distributed import Client
from distributed.utils import format_bytes
client = Client(cluster)
cluster

In [ ]:
cluster.close()

### Zarr-related Helper Functions

In [ ]:
### Preprocessing Steps for each input dataset before merge
def preprocess_sparse(ds):
    """Pare down each input dataset to a single variable.  
       The subsequent merge will eliminate unused coordinates automatically. 
        
        This function does not allow additional arguments, so the target 
        output variable needs to be defined globally in TARGET_VAR.
    """
    drop_vars = [var for var in ds.data_vars 
                 if var != TARGET_VAR]

    ds_fixed = ds.drop_vars(drop_vars)
    
    lats = list(ds.pfts1d_lat.astype('float32').data)
    lons = list(ds.pfts1d_lon.astype('float32').data)
    vegtype = list(ds.pfts1d_itype_veg.data)
    coltype = list(ds.pfts1d_itype_col.data)
    lunittype = list(ds.pfts1d_itype_lunit.data)
    active = list(ds.pfts1d_active.data)
    
    # Redefine the 'pft' dimension as a multi-index, which will increase the number of dimensions.
    #arrays = [list(lats.data), list(lons.data), list(vegtype.data), list(coltype.data), list(lunittype.data, active.data]
    #dim_names = ('pftlat', 'pftlon', 'vegtype', 'coltype', 'lunittype', 'active')
    index = pd.MultiIndex.from_arrays([lats, lons, vegtype, coltype, lunittype, active], 
                                  names=('pftlat', 'pftlon', 'vegtype', 'coltype', 'lunittype', 'active'))
    ds_fixed['pft'] = index

    
    # Keep the data sparse if possible to avoid memory shortages.
    ds_fixed = ds_fixed.unstack(sparse=True)
    return ds_fixed

In [4]:
### Preprocessing Steps for each input dataset before merge
def preprocess(ds):
    """Pare down each input dataset to a single variable.  
       The subsequent merge will eliminate unused coordinates automatically. 
        
        This function does not allow additional arguments, so the target 
        output variable needs to be defined globally in TARGET_VAR.
    """
    drop_vars = [var for var in ds.data_vars 
                 if var != TARGET_VAR]

    ds_fixed = ds.drop_vars(drop_vars)
    
    # Drop these unused dimensions if they exist.
    drop_dims = ['levlak', 'levgrnd', 'levdcmp']
    to_drop = [d for d in drop_dims if d in ds_fixed.dims]
    ds_fixed = ds_fixed.drop_dims(to_drop)

    return ds_fixed

In [5]:
def print_ds_info(ds, var):
    """Function for printing chunking information"""

    print(f'print_ds_info: var == {var}')
    dt = ds[var].dtype
    itemsize = dt.itemsize
    chunk_size = ds[var].data.chunksize
    size = format_bytes(ds.nbytes)
    _bytes = reduce(mul, chunk_size) * itemsize
    chunk_size_bytes = format_bytes(_bytes)

    print(f'Variable name: {var}')
    print(f'Dataset dimensions: {ds[var].dims}')
    print(f'Chunk shape: {chunk_size}')
    print(f'Dataset shape: {ds[var].shape}')
    print(f'Chunk size: {chunk_size_bytes}')
    print(f'Dataset size: {size}')

    
def zarr_store(var, dirout, write=False):
    """ Create zarr store name/path
    """
    path = f'{dirout}/{var}.zarr'
    if write and os.path.exists(path):
        shutil.rmtree(path)
    print(path)
    return path


def save_data(ds, store):
    try:
        ds.to_zarr(store=store, consolidated=True)
        del ds
    except Exception as e:
        print(f"Failed to write {store}: {e}")

        
def zarr_check():
    '''Make sure the zarr stores were properly written'''

    from pathlib import Path
    p = Path(output_folder)
    stores = list(p.rglob("*.zarr"))
    #stores = list(p.rglob("*.rcp45.day.NAM-22i.raw.zarr"))
    for store in stores:
        try:
            ds = xr.open_zarr(store.as_posix(), consolidated=True)
            print('\n')
            print(store)
            print(ds)
        except Exception as e:
            #print(e)
            print(store)

In [9]:
# Open catalog with single-valued "variable" column

# Have the catalog interpret the "variable" column as a list of values.
col = intake.open_esm_datastore(catalog_path)
col


,unique
variable,4
member_id,80
path,80


In [10]:
# Show the eventual output store base names.
print("Eventual store base names:")
print(col.keys())

Eventual store base names:
dict_keys(['EFLX_LH_TOT', 'ER', 'HR', 'TSA'])


In [14]:
REALLY_SAVE = True

for variable in output_variables:
    # This variable gets used in the "preprocess" function and must be defined now in the global scope.
    TARGET_VAR = variable

    col_subset = col.search(variable = variable)
    # Produce var-based stores.  The catalog will determine how many stores and their base names.
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        dsets = col_subset.to_dataset_dict(zarr_kwargs={'consolidated': True}, preprocess=preprocess)

    ds_out = dsets[variable]
    
    # Specify final chunking.
    ds_out = ds_out.chunk(target_chunks)

    # Update store metadata
    ds_out.attrs = update_metadata(ds_out.attrs)

    
    # Confirm output contents.
    print_ds_info(ds_out, variable)
    
    
    store = zarr_store(variable, dirout = output_folder, write=REALLY_SAVE)
    if REALLY_SAVE:
        save_data(ds_out, store=store)
        print("     ... Done.")
    else:
        print("     ... (Skipping)")
        del ds_out



--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable'


print_ds_info: var == ER
Variable name: ER
Dataset dimensions: ('member_id', 'time', 'lat', 'lon')
Chunk shape: (40, 800, 32, 32)
Dataset shape: (80, 11687, 192, 288)
Chunk size: 125.00 MiB
Dataset size: 192.60 GiB
/glade/scratch/bonnland/DART/ds345.0/zarr-publish/ER.zarr
     ... Done.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable'


print_ds_info: var == EFLX_LH_TOT
Variable name: EFLX_LH_TOT
Dataset dimensions: ('member_id', 'time', 'lat', 'lon')
Chunk shape: (40, 800, 32, 32)
Dataset shape: (80, 11687, 192, 288)
Chunk size: 125.00 MiB
Dataset size: 192.60 GiB
/glade/scratch/bonnland/DART/ds345.0/zarr-publish/EFLX_LH_TOT.zarr
     ... Done.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable'


print_ds_info: var == HR
Variable name: HR
Dataset dimensions: ('member_id', 'time', 'lat', 'lon')
Chunk shape: (40, 800, 32, 32)
Dataset shape: (80, 11687, 192, 288)
Chunk size: 125.00 MiB
Dataset size: 192.60 GiB
/glade/scratch/bonnland/DART/ds345.0/zarr-publish/HR.zarr
     ... Done.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable'


print_ds_info: var == TSA
Variable name: TSA
Dataset dimensions: ('member_id', 'time', 'lat', 'lon')
Chunk shape: (40, 800, 32, 32)
Dataset shape: (80, 11687, 192, 288)
Chunk size: 125.00 MiB
Dataset size: 192.60 GiB
/glade/scratch/bonnland/DART/ds345.0/zarr-publish/TSA.zarr
     ... Done.


In [15]:
# Open each output dataset to confirm it was created properly.

zarr_check()



/glade/scratch/bonnland/DART/ds345.0/zarr-publish/HR.zarr
<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, member_id: 80, time: 11687)
Coordinates:
  * lat        (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 9 ... 73 74 75 76 77 78 79 80
  * time       (time) datetime64[ns] 2012-01-01T06:00:00 ... 2019-12-31T18:00:00
Data variables:
    HR         (member_id, time, lat, lon) float32 dask.array<chunksize=(40, 800, 32, 32), meta=np.ndarray>
Attributes: (12/16)
    Initial_conditions_dataset:           f.e21.FHIST_BGC.f09_025.CAM6assim.0...
    PFT_physiological_constants_dataset:  clm5_params.c171117.nc
    Surface_dataset:                      surfdata_0.9x1.25_78pfts_CMIP6_simy...
    case_id:                              f.e21.FHIST_BGC.f09_025.CAM6assim.011
    cft:                                  {"cft_irrigated_oilpalm": 40, "cft_..

In [16]:
!date

ValueError: filedescriptor out of range in select()

In [ ]:
cluster.close()

/glade/work/bonnland/miniconda3/envs/lens-conversion/lib/python3.8/site-packages/dask_jobqueue/core.py:360: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone
/glade/work/bonnland/miniconda3/envs/lens-conversion/lib/python3.8/site-packages/dask_jobqueue/core.py:360: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone
/glade/work/bonnland/miniconda3/envs/lens-conversion/lib/python3.8/site-packages/dask_jobqueue/core.py:360: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # del